In [1]:
# Package Declaration
# !pip install rasterio
!pip install netCDF4
!pip install geopandas

# from osgeo import gdal
# from osgeo import ogr
# from osgeo import osr
# from osgeo import gdal_array
# from osgeo import gdalconst

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import netCDF4 as nc

# import rasterio

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 1.0MB 22.5MB/s 
     |████████████████████████████████| 15.3MB 230kB/s 
     |████████████████████████████████| 6.5MB 34.9MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0
Mounted at /content/gdrive


In [2]:
# DATABOX
bound_w	= 28.417
bound_e	= 41.194
bound_n =	3.622
bound_s =	-12.183

buffer_grid = 1

In [6]:
#CHIRPS 05
fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2017.days_p05.nc"
data = nc.Dataset(fname,'r')

lats = data.variables['latitude'][:]
lons = data.variables['longitude'][:]

lat_bnds, lon_bnds = [bound_s, bound_n], [bound_w, bound_e]

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lat_inds = np.asarray(lat_inds)

lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
lon_inds = np.asarray(lon_inds)

lat_list = data.variables['latitude'][lat_inds[0,0]:lat_inds[0,-1]]
lon_list = data.variables['longitude'][lon_inds[0,0]:lon_inds[0,-1]]

ds = xr.open_dataset(fname)
ds_clip = ds.sel(latitude=lat_list, longitude=lon_list, method='nearest')

lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# print(lat_interp)
# print(lon_interp)

ds_clip_interp = ds_clip.interp(latitude=lat_interp, longitude=lon_interp, method='nearest')
print(ds_clip_interp)

# print(ds_clip)
# print(ds_clip_interp)

<xarray.Dataset>
Dimensions:    (latitude: 314, longitude: 254, time: 365)
Coordinates:
  * time       (time) datetime64[ns] 2017-01-01 2017-01-02 ... 2017-12-31
  * latitude   (latitude) float64 -12.17 -12.12 -12.07 ... 3.375 3.425 3.475
  * longitude  (longitude) float64 28.43 28.48 28.53 28.58 ... 40.98 41.03 41.08
Data variables:
    precip     (time, latitude, longitude) float32 22.98 23.23 0.0 ... 0.0 0.0
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2018-01-17
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given day....
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org

In [ ]:
# #CHIRPS Monthly
# fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/monthly/chirps-v2.0.monthly.nc"
# data = nc.Dataset(fname,'r')

# lats = data.variables['latitude'][:]
# lons = data.variables['longitude'][:]

# lat_bnds, lon_bnds = [-11.183, 3.622], [28.417, 41.694]

# lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
# lat_inds = np.asarray(lat_inds)

# lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
# lon_inds = np.asarray(lon_inds)

# lat_list = data.variables['latitude'][lat_inds[0,0]:lat_inds[0,-1]]
# lon_list = data.variables['longitude'][lon_inds[0,0]:lon_inds[0,-1]]

# ds = xr.open_dataset(fname)
# ds_clip = ds.sel(latitude=lat_list, longitude=lon_list, method='nearest')

# lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
# lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# print(lat_interp)
# print(lon_interp)

# ds_clip_interp = ds_clip.interp(latitude=lat_interp, longitude=lon_interp, method='nearest')

# print(ds_clip)
# print(ds_clip_interp)

[-11.17499924 -11.16499924 -11.15499924 ...   3.49500076   3.50500076
   3.51500076]
[28.42500305 28.43500305 28.44500305 ... 41.59500305 41.60500305
 41.61500305]
<xarray.Dataset>
Dimensions:    (latitude: 295, longitude: 265, time: 482)
Coordinates:
  * latitude   (latitude) float32 -11.174999 -11.125 ... 3.4749985 3.5249977
  * longitude  (longitude) float32 28.425003 28.47499 ... 41.574997 41.625
  * time       (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2021-02-01
Data variables:
    precip     (time, latitude, longitude) float32 ...
Attributes:
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2021-03-16
    creator_name:      Pete Peterson
    creator_email:     pete@geog.ucsb.edu
    institution:       Climate Hazards Group.  University of California at Sa...
    documentation:     http://pubs.usgs.gov/ds/832/
    reference:         Funk, C

In [12]:
# #CMORPH DAILY
# fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/daily/CMORPH/CMORPH_V1.0_ADJ_0.25deg-DLY_00Z_20191030.nc"
# data = nc.Dataset(fname,'r')

# lats = data.variables['lat'][:]
# lons = data.variables['lon'][:]

# lat_bnds, lon_bnds = [bound_s, bound_n], [bound_w, bound_e]

# lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
# lat_inds = np.asarray(lat_inds)

# lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
# lon_inds = np.asarray(lon_inds)

# lat_list = data.variables['lat'][lat_inds[0,0]:lat_inds[0,-1]]
# lon_list = data.variables['lon'][lon_inds[0,0]:lon_inds[0,-1]]

# ds = xr.open_dataset(fname)
# ds_clip = ds.sel(lat=lat_list, lon=lon_list, method='nearest')

# lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
# lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# # print(lat_interp)
# # print(lon_interp)

# ds_clip_interp = ds_clip.interp(lat=lat_interp, lon=lon_interp, method='nearest')

# # print(ds_clip)
# print(ds_clip_interp)

<xarray.Dataset>
Dimensions:      (lat: 305, lon: 245, nv: 2, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 1998-01-01
  * lat          (lat) float64 -12.12 -12.07 -12.02 -11.97 ... 2.975 3.025 3.075
  * lon          (lon) float64 28.62 28.68 28.73 28.78 ... 40.73 40.78 40.83
Dimensions without coordinates: nv
Data variables:
    time_bounds  (time, nv) datetime64[ns] 1998-01-01 1998-01-01T23:59:59
    lat_bounds   (lat, nv) float32 -12.25 -12.0 -12.25 -12.0 ... 3.25 3.0 3.25
    lon_bounds   (lon, nv) float32 28.5 28.75 28.5 28.75 ... 41.0 40.75 41.0
    cmorph       (time, lat, lon) float32 18.7 18.7 18.7 16.7 ... 28.6 26.0 26.0
Attributes: (12/57)
    ncei_template_version:      NCEI_NetCDF_Grid_template_V2.0
    title:                      NOAA Climate Data Record (CDR) of CPC Morphin...
    keywords:                   Precipitation, Satellite, High-Resolution, Gl...
    summary:                    The CMORPH CDR is a reprocessed and bias-corr...
    references:     

In [18]:
# #CMORPH DAILY AGG
# fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/daily_agg/daily_aggCMORPH_V1.0_ADJ_8km-30min_20160101.nc"
# data = nc.Dataset(fname,'r')

# print(data)

# lats = data.variables['Latitude'][:]
# lons = data.variables['Longitude'][:]

# lat_bnds, lon_bnds = [bound_s, bound_n], [bound_w, bound_e]

# lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
# lat_inds = np.asarray(lat_inds)

# lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
# lon_inds = np.asarray(lon_inds)

# lat_list = data.variables['Latitude'][lat_inds[0,0]:lat_inds[0,-1]]
# lon_list = data.variables['Longitude'][lon_inds[0,0]:lon_inds[0,-1]]

# ds = xr.open_dataset(fname)
# ds_clip = ds.sel(lat=lat_list, lon=lon_list, method='nearest')

# lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
# lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# print(lat_interp)
# print(lon_interp)

# ds_clip_interp = ds_clip.interp(latitude=lat_interp, longitude=lon_interp, method='nearest')

# print(ds_clip)
# print(ds_clip_interp)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(181), lat(202), time(1)
    variables(dimensions): float32 Longitude(lon), float32 Latitude(lat), float32 precip(time, lat, lon), int32 time(time)
    groups: 


ValueError: ignored

In [24]:
#IMERG
fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/3B-DAY-L.MS.MRG.3IMERG.20210402-S000000-E235959.V06.nc4"
data = nc.Dataset(fname,'r')

lats = data.variables['lat'][:]
lons = data.variables['lon'][:]

lat_bnds, lon_bnds = [bound_s, bound_n], [bound_w, bound_e]

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lat_inds = np.asarray(lat_inds)

lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
lon_inds = np.asarray(lon_inds)

lat_list = data.variables['lat'][lat_inds[0,0]:lat_inds[0,-1]]
lon_list = data.variables['lon'][lon_inds[0,0]:lon_inds[0,-1]]

ds = xr.open_dataset(fname)
ds_clip = ds.sel(lat=lat_list, lon=lon_list, method='nearest')

lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# print(lat_interp)
# print(lon_interp)

ds_clip_interp = ds_clip.interp(lat=lat_interp, lon=lon_interp, method='nearest')

# print(ds_clip)
print(ds_clip_interp)

<xarray.Dataset>
Dimensions:                    (lat: 312, lon: 252, nv: 2, time: 1)
Coordinates:
  * time                       (time) object 2021-04-02 00:00:00
  * lat                        (lat) float64 -12.15 -12.1 -12.05 ... 3.35 3.4
  * lon                        (lon) float64 28.45 28.5 28.55 ... 40.95 41.0
Dimensions without coordinates: nv
Data variables:
    precipitationCal           (time, lon, lat) float32 0.0 0.0 0.0 ... 0.0 0.0
    precipitationCal_cnt       (time, lon, lat) float64 48.0 48.0 ... 48.0 48.0
    precipitationCal_cnt_cond  (time, lon, lat) float64 0.0 0.0 0.0 ... 0.0 0.0
    HQprecipitation            (time, lon, lat) float32 0.0 0.0 0.0 ... 0.0 0.0
    HQprecipitation_cnt        (time, lon, lat) float64 7.0 7.0 7.0 ... 9.0 9.0
    HQprecipitation_cnt_cond   (time, lon, lat) float64 0.0 0.0 0.0 ... 0.0 0.0
    randomError                (time, lon, lat) float32 0.8249 0.8249 ... 0.8249
    randomError_cnt            (time, lon, lat) float64 48.0 48.0 ... 

In [30]:
#TAMSAT DAILY
fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/TAMSAT/daily/01-tamsatDaily.v3.1-410227200-1617231600_uga.nc"
data = nc.Dataset(fname,'r')

lats = data.variables['lat'][:]
lons = data.variables['lon'][:]

lat_bnds, lon_bnds = [bound_s, bound_n], [bound_w, bound_e]

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lat_inds = np.asarray(lat_inds)

lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
lon_inds = np.asarray(lon_inds)

lat_list = data.variables['lat'][lat_inds[0,0]:lat_inds[0,-1]]
lon_list = data.variables['lon'][lon_inds[0,0]:lon_inds[0,-1]]

ds = xr.open_dataset(fname)
ds_clip = ds.sel(lat=lat_list, lon=lon_list, method='nearest')

lat_interp = np.arange(lat_list[0],lat_list[-1],0.05)
lon_interp = np.arange(lon_list[0],lon_list[-1],0.05)

# print(lat_interp)
# print(lon_interp)

ds_clip_interp = ds_clip.interp(lat=lat_interp, lon=lon_interp, method='nearest')

# print(ds_clip)
print(ds_clip_interp)

ValueError: ignored